In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [71]:
import pandas as pd
import requests

from concurrent.futures import ThreadPoolExecutor, as_completed

In [72]:
#reading from base dataset and standardising the column names
data_df = pd.read_csv('/content/drive/MyDrive/DSP Project Data/Electric_Vehicle__EV__Charging_Data-_Municipal_Lots_and_Garages.csv')

data_df.rename(columns = {'Date':'date','Station Name': 'station_name','Location Name':'location_name','Country':'country','Charge Box ID':'charge_box_id',
                          'Connector ID':'connector_id','Driver ID':'driver_id','ID Tag':'id_tag','Connected Time':'connected_time','Disconnected Time':'disconnected_time',
                          'Charge Duration (min)':'charge_duration_min','Connected Duration (min)':'connected_duration_min','Energy Provided (kWh)':'energy_provided_kwh',
                          'Session Status':'session_status','Invalidity Reason':'invalidity_reason'}, inplace = True)

In [73]:
data_df.head(3)

,date,station_name,location_name,country,charge_box_id,connector_id,driver_id,id_tag,connected_time,disconnected_time,charge_duration_min,connected_duration_min,energy_provided_kwh,session_status,invalidity_reason
0,08/21/2025,101013,JGU - Jerome Gun Hill Road Municipal Parking G...,USA,101013,1,4a37c773-2997-472d-a5e1-0dfc26bf4883,ADJRKBZ9BBMW0B0LMVCA,19:10:54.0000000,07:11:12.0000000,720.300000,720.300000,41.369,PAID,NaN
1,08/21/2025,101014,JGU - Jerome Gun Hill Road Municipal Parking G...,USA,101014,1,7fa6b416-4196-4120-b6bd-409e1ad65ba6,78I78199YN3TRVI1OSMH,16:53:00.0000000,20:29:07.0000000,160.100000,216.116667,15.814,PAID,NaN
2,08/21/2025,101016,JGU - Jerome Gun Hill Road Municipal Parking G...,USA,101016,1,97b09152-a9c2-4113-a7ab-aef174f958ca,V7IA0NBZQ3POKCQIYSRR,23:08:38.0000000,09:33:03.0000000,570.666667,624.416667,31.867,PAID,NaN


In [74]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206897 entries, 0 to 206896
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date                    206897 non-null  object 
 1   station_name            206897 non-null  object 
 2   location_name           206868 non-null  object 
 3   country                 126438 non-null  object 
 4   charge_box_id           206897 non-null  object 
 5   connector_id            206897 non-null  int64  
 6   driver_id               196741 non-null  object 
 7   id_tag                  206897 non-null  object 
 8   connected_time          206897 non-null  object 
 9   disconnected_time       206897 non-null  object 
 10  charge_duration_min     206897 non-null  float64
 11  connected_duration_min  206897 non-null  float64
 12  energy_provided_kwh     206897 non-null  float64
 13  session_status          206897 non-null  object 
 14  invalidity_reason   

In [75]:
#Converting date columns to pandas timestamp format
data_df.date = pd.to_datetime(data_df.date)
data_df['connected_time_start_ts'] = pd.to_datetime(data_df.date.dt.strftime('%Y-%m-%d') + ' ' + data_df.connected_time.str[:8])

#Dropping columns with no values
data_df.drop(columns = ['invalidity_reason'], inplace=True)

#Very small portion (less than 0.01%) of location_name is null, hence dropping those rows
data_df.dropna(subset=['location_name'], inplace=True)

#Filling NA values
data_df.fillna({'country':'USA','driver_id': 'UNKNOWN'}, inplace=True)

#Standarizing string columns
cols_to_upper = ["station_name", "location_name", "country",'charge_box_id','driver_id','id_tag','session_status']

for col in cols_to_upper:
    data_df[col] = data_df[col].str.upper()

In [76]:
data_df.location_name.unique()

array(['JGU - JEROME GUN HILL ROAD MUNICIPAL PARKING GARAGE',
       'BRI - BAY RIDGE MUNICIPAL PARKING GARAGE',
       'DES - DELANCEY AND ESSEX MUNICIPAL PARKING GARAGE',
       'QBO - QUEENS BOROUGH HALL MUNICIPAL PARKING GARAGE',
       'JON - JEROME 190TH STREET MUNICIPAL PARKING',
       'SGE - ST. GEORGE COURTHOUSE GARAGE', 'ST. GEORGE COURTHOUSE',
       'CSQ - COURT SQUARE MUNICIPAL PARKING GARAGE',
       'QFA - QUEENS FAMILY COURT MUNICIPAL GARAGE',
       'DELANCEY AND ESSEX MUNICIPAL PARKING GARAGE',
       'COURT SQUARE MUNICIPAL PARKING GARAGE', 'QUEENSBORO HALL',
       'JEROME 190TH STREET MUNICIPAL PARKING',
       'BAY RIDGE MUNICIPAL PARKING GARAGE',
       'QUEENS FAMILY COURT MUNICIPAL GARAGE',
       'QUEENS BOROUGH HALL MUNICIPAL PARKING GARAGE',
       'JEROME-GUN HILL ROAD MUNICIPAL PARKING GARAGE',
       'JON - JEROME 190TH STREET MUNICIPAL PARKING GARAGE',
       'SGE - ST. GEORGE COURTHOUSE MUNICIPAL PARKING GARAGE'],
      dtype=object)

In [77]:
#Based on observation, same garage has been represneted in more than 1 way. Changing them to single location
data_df.loc[data_df.location_name.str.contains('ST. GEORGE COURTHOUSE'), 'location_name'] = 'SGE - ST. GEORGE COURTHOUSE MUNICIPAL PARKING GARAGE'
data_df.loc[data_df.location_name.str.contains('QUEENSBORO HALL'), 'location_name'] = 'COURT SQUARE MUNICIPAL PARKING GARAGE'

In [78]:
#Adding a unique identifer for each row to merge with address details
mapping = {
    'JGU - JEROME GUN HILL ROAD MUNICIPAL PARKING GARAGE':'GARAGE_1',
    'BRI - BAY RIDGE MUNICIPAL PARKING GARAGE':'GARAGE_2',
    'DES - DELANCEY AND ESSEX MUNICIPAL PARKING GARAGE':'GARAGE_3',
    'QBO - QUEENS BOROUGH HALL MUNICIPAL PARKING GARAGE':'GARAGE_4',
    'JON - JEROME 190TH STREET MUNICIPAL PARKING':'GARAGE_5',
    'SGE - ST. GEORGE COURTHOUSE MUNICIPAL PARKING GARAGE':'GARAGE_6',
    'CSQ - COURT SQUARE MUNICIPAL PARKING GARAGE':'GARAGE_7',
    'QFA - QUEENS FAMILY COURT MUNICIPAL GARAGE':'GARAGE_8',
    'DELANCEY AND ESSEX MUNICIPAL PARKING GARAGE':'GARAGE_9',
    'COURT SQUARE MUNICIPAL PARKING GARAGE':'GARAGE_10',
    'JEROME 190TH STREET MUNICIPAL PARKING':'GARAGE_11',
    'BAY RIDGE MUNICIPAL PARKING GARAGE':'GARAGE_12',
    'QUEENS FAMILY COURT MUNICIPAL GARAGE':'GARAGE_13',
    'QUEENS BOROUGH HALL MUNICIPAL PARKING GARAGE':'GARAGE_14',
    'JEROME-GUN HILL ROAD MUNICIPAL PARKING GARAGE':'GARAGE_15',
    'JON - JEROME 190TH STREET MUNICIPAL PARKING GARAGE':'GARAGE_16'
}

data_df['location_id'] = data_df['location_name'].map(mapping)

In [79]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 206868 entries, 0 to 206896
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   date                     206868 non-null  datetime64[ns]
 1   station_name             206868 non-null  object        
 2   location_name            206868 non-null  object        
 3   country                  206868 non-null  object        
 4   charge_box_id            206868 non-null  object        
 5   connector_id             206868 non-null  int64         
 6   driver_id                206868 non-null  object        
 7   id_tag                   206868 non-null  object        
 8   connected_time           206868 non-null  object        
 9   disconnected_time        206868 non-null  object        
 10  charge_duration_min      206868 non-null  float64       
 11  connected_duration_min   206868 non-null  float64       
 12  energy_provided_kwh  

In [80]:
#Reading address info for each garage
garage_df = pd.read_csv('/content/drive/MyDrive/DSP Project Data/garage_address_info.csv')

In [81]:
garage_df.head()

,id,location_name,zip_code,latitude,longitude,locality,state
0,garage_1,jgu - jerome gun hill road municipal parking g...,10467,40.8852,-73.8795,Bronx,NYC
1,garage_2,bri - bay ridge municipal parking garage,11209,40.6300,-74.0260,Manhattan,NYC
2,garage_3,des - delancey and essex municipal parking garage,10002,40.7178,-73.9874,Manhattan,NYC
3,garage_4,qbo - queens borough hall municipal parking ga...,11415,40.7110,-73.8238,Kew Gardens,NYC
4,garage_5,jon - jerome 190th street municipal parking,10468,40.8852,-73.8795,Bronx,NYC


In [82]:
#Standardizing string columns
cols_to_upper = ["id", "location_name", "locality",'state']

for col in cols_to_upper:
    garage_df[col] = garage_df[col].str.upper()

In [83]:
#Merging address details with main dataframe
data_df = pd.merge(data_df, garage_df, on='location_name', how='left')
data_df.drop(columns=['location_id'], inplace=True)

In [84]:
data_df.head(2)

,date,station_name,location_name,country,charge_box_id,connector_id,driver_id,id_tag,connected_time,disconnected_time,...,connected_duration_min,energy_provided_kwh,session_status,connected_time_start_ts,id,zip_code,latitude,longitude,locality,state
0,2025-08-21,101013,JGU - JEROME GUN HILL ROAD MUNICIPAL PARKING G...,USA,101013,1,4A37C773-2997-472D-A5E1-0DFC26BF4883,ADJRKBZ9BBMW0B0LMVCA,19:10:54.0000000,07:11:12.0000000,...,720.300000,41.369,PAID,2025-08-21 19:10:54,GARAGE_1,10467,40.8852,-73.8795,BRONX,NYC
1,2025-08-21,101014,JGU - JEROME GUN HILL ROAD MUNICIPAL PARKING G...,USA,101014,1,7FA6B416-4196-4120-B6BD-409E1AD65BA6,78I78199YN3TRVI1OSMH,16:53:00.0000000,20:29:07.0000000,...,216.116667,15.814,PAID,2025-08-21 16:53:00,GARAGE_1,10467,40.8852,-73.8795,BRONX,NYC


In [85]:
#Creating a new dataframe with distinct latitude, longitude, date for extracting weather info
location_time_df = data_df[['date','latitude','longitude']].copy()
location_time_df.drop_duplicates(inplace=True)
location_time_df.reset_index(drop=True, inplace=True)

location_time_df['temp'] = ''
location_time_df['tempmax'] = ''
location_time_df['tempmin'] = ''
location_time_df['humidity'] = ''
location_time_df['conditions'] = ''

In [27]:
#Function to get weather data via VisualCrossing API
def get_weather(lat, lon, date, api_key):
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat},{lon}/{date}?key={api_key}"
    response = requests.get(url)
    data = response.json()

    # Extract key info
    day = data["days"][0]
    return {
        "date": day["datetime"],
        "temp": day["temp"],
        "tempmax": day["tempmax"],
        "tempmin": day["tempmin"],
        "humidity": day["humidity"],
        "conditions": day["conditions"]
    }

In [ ]:
#Running concurrently to fetch data faster
def fetch_row(row):
    try:
        return row.Index, get_weather(
            row.latitude,
            row.longitude,
            row.date.strftime('%Y-%m-%d'),
            "6AZHKUT6JDZBWMUYC74U4DZUC"
        )
    except Exception:
        return row.Index, None

with ThreadPoolExecutor(max_workers=120) as executor:
    futures = {
        executor.submit(fetch_row, row): row
        for row in location_time_df.itertuples()
    }

    for fut in as_completed(futures):
        idx, resp = fut.result()
        if resp is None:
            continue

        location_time_df.at[idx, 'temp'] = resp['temp']
        location_time_df.at[idx, 'tempmax'] = resp['tempmax']
        location_time_df.at[idx, 'tempmin'] = resp['tempmin']
        location_time_df.at[idx, 'humidity'] = resp['humidity']
        location_time_df.at[idx, 'conditions'] = resp['conditions']


In [86]:
#As the weather extraction is limited by API limits, the data has been extracted once and stored as a csv file
#Reading the csv file for weather
location_time_df = pd.read_csv('/content/drive/MyDrive/DSP Project Data/location_time_weather_info.csv')
location_time_df.date = pd.to_datetime(location_time_df.date)

In [87]:
location_time_df.head()

,date,latitude,longitude,temp,tempmax,tempmin,humidity,conditions
0,2025-08-21,40.8852,-73.8795,64.8,71.6,59.5,74.4,"Rain, Partially cloudy"
1,2025-08-21,40.6300,-74.0260,64.7,72.7,59.7,75.1,"Rain, Partially cloudy"
2,2025-08-21,40.7178,-73.9874,64.4,71.4,59.4,75.2,"Rain, Partially cloudy"
3,2025-08-21,40.7110,-73.8238,65.2,72.4,60.4,75.3,"Rain, Partially cloudy"
4,2025-08-21,37.1050,-113.5860,94.5,106.1,80.9,18.6,Clear


In [88]:
#Merging weather info to create final dataframe
data_df = pd.merge(data_df, location_time_df, on=['date','latitude','longitude'], how='left')

In [89]:
data_df.head(3)

,date,station_name,location_name,country,charge_box_id,connector_id,driver_id,id_tag,connected_time,disconnected_time,...,zip_code,latitude,longitude,locality,state,temp,tempmax,tempmin,humidity,conditions
0,2025-08-21,101013,JGU - JEROME GUN HILL ROAD MUNICIPAL PARKING G...,USA,101013,1,4A37C773-2997-472D-A5E1-0DFC26BF4883,ADJRKBZ9BBMW0B0LMVCA,19:10:54.0000000,07:11:12.0000000,...,10467,40.8852,-73.8795,BRONX,NYC,64.8,71.6,59.5,74.4,"Rain, Partially cloudy"
1,2025-08-21,101014,JGU - JEROME GUN HILL ROAD MUNICIPAL PARKING G...,USA,101014,1,7FA6B416-4196-4120-B6BD-409E1AD65BA6,78I78199YN3TRVI1OSMH,16:53:00.0000000,20:29:07.0000000,...,10467,40.8852,-73.8795,BRONX,NYC,64.8,71.6,59.5,74.4,"Rain, Partially cloudy"
2,2025-08-21,101016,JGU - JEROME GUN HILL ROAD MUNICIPAL PARKING G...,USA,101016,1,97B09152-A9C2-4113-A7AB-AEF174F958CA,V7IA0NBZQ3POKCQIYSRR,23:08:38.0000000,09:33:03.0000000,...,10467,40.8852,-73.8795,BRONX,NYC,64.8,71.6,59.5,74.4,"Rain, Partially cloudy"


In [90]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206868 entries, 0 to 206867
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   date                     206868 non-null  datetime64[ns]
 1   station_name             206868 non-null  object        
 2   location_name            206868 non-null  object        
 3   country                  206868 non-null  object        
 4   charge_box_id            206868 non-null  object        
 5   connector_id             206868 non-null  int64         
 6   driver_id                206868 non-null  object        
 7   id_tag                   206868 non-null  object        
 8   connected_time           206868 non-null  object        
 9   disconnected_time        206868 non-null  object        
 10  charge_duration_min      206868 non-null  float64       
 11  connected_duration_min   206868 non-null  float64       
 12  energy_provided_

In [91]:
#Sorting data to populate lag columns
data_df.sort_values(['driver_id', 'station_name', 'connected_time_start_ts'], inplace=True)
data_df.connected_time_start_ts = pd.to_datetime(data_df.connected_time_start_ts)

data_df['charge_duration_hours'] = round(data_df.charge_duration_min / 60,0)

data_df['last_date'] = data_df.groupby(['driver_id','station_name'])['connected_time_start_ts'].shift(1)
data_df['days_between_charges'] = (data_df.connected_time_start_ts - data_df.last_date).dt.days

In [92]:
#Mapping hours to categories
mapping = {
    '00' : 'late_night',
    '01' : 'late_night',
    '02' : 'late_night',
    '03' : 'late_night',
    '04' : 'late_night',
    '05' : 'early_morning',
    '06' : 'early_morning',
    '07' : 'morning',
    '08' : 'morning',
    '09' : 'morning',
    '10' : 'morning',
    '11' : 'morning',
    '12' : 'afternoon',
    '13' : 'afternoon',
    '14' : 'afternoon',
    '15' : 'afternoon',
    '16' : 'afternoon',
    '17' : 'evening',
    '18' : 'evening',
    '19' : 'evening',
    '20' : 'night',
    '21' : 'night',
    '22' : 'night',
    '23' : 'late_night'
}

data_df['connected_slot'] = data_df.connected_time.str[:2].map(mapping)
data_df['disconnected_slot'] = data_df.disconnected_time.str[:2].map(mapping)

In [93]:
#Dropping unwanted columns
cols_to_drop = ['country', 'charge_box_id', 'connector_id', 'id_tag', 'connected_time', 'disconnected_time', 'id','latitude','longitude','zip_code','state','last_date']
data_df.drop(columns=cols_to_drop, inplace=True)

data_df.days_between_charges = data_df.days_between_charges.fillna(0)

In [97]:
data_df.to_csv('EV_Charging_Data_NYC_Enhanced.csv', index = False)

In [70]:
#End of data collection and pre-processing